Our Objective is to predict GDP of the country

The data definition is as follows:
Country,Population of the country,Area_sqm,Population_Density_Per_sqm,Coastline (coast/area ratio), Country,Population,Area_sqm,Pop_Density_per sqm,,Net migration,Infant mortality (per 1000 births),
GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,
Industry,Service.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/tkseneee/Dataset/master/GDP_Country1.csv')
data.head()

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:
# Step - 1---> Preprocessing:

In [ ]:
dd = data.set_index('Country')
dd.head()

In [ ]:
# Null value Imputation:

In [ ]:
dd.isnull().sum()

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
impu = KNNImputer()

dt = impu.fit_transform(dd)

dt = pd.DataFrame(dt,columns = dd.columns, index = dd.index)
dt.head()

In [ ]:
dt.isnull().sum()

In [ ]:
# To check Outlier:

for i in dt.columns:
    dt[i].plot(kind = 'box')
    plt.show()

In [ ]:
# Outlier Treatment:

# by capping:

for i in dt.columns:
    q1=dt[i].quantile(0.25)
    q3=dt[i].quantile(0.75)
    iqr=q3-q1
    ub=q3 + 1.5*iqr
    lb=q1 - 1.5*iqr
    uc=dt[i].quantile(0.99)
    lc=dt[i].quantile(0.01)
    for ind1 in dt[i].index:
        if dt.loc[ind1, i] >ub:            
            dt.loc[ind1, i] =uc
        if dt.loc[ind1, i] < lb:
            dt.loc[ind1, i] =lc

In [ ]:
# To recheck the outlier treatment:

for i in dt.columns:
    dt[i].plot(kind = 'box')
    plt.show()

In [ ]:
# Step - 2 -----> To check whether the overall model is significant or not?

inp = dt.drop('GDP ($ per capita)',axis = 1)
out = dt['GDP ($ per capita)']

inpc = sm.add_constant(inp)

ols = sm.OLS(out,inpc)
ols_mod = ols.fit()
ols_mod.summary()

In [ ]:
# Inference: From the above model summary of Probability of fstat(p < 0.05), we can conclude that overall model is significant.

# From r-square value we can infere that, model ability to explain target variation with respect to input is 80%.
# Bias error can be calculated as 1 - 0.801 is approximatley equal to 0.2. i.e 20%.

In [ ]:
# Step - 3 ----> To proceed with linear regression model, 5 assumptions need to be checked.

# 1. Multicollinearity
# 2. Normality
# 3. Linearity
# 4. Auto-correlation
# 5. Homoscadacity

In [ ]:
# 1. To check multicollinearity:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
inp1 = inp - inp.mean()

In [ ]:
inp1c = sm.add_constant(inp1)

ols = sm.OLS(out,inp1c)
ols_mod = ols.fit()
ols_mod.summary()

In [ ]:
# To check variance-inflation-factor:

vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(inp1c.values,i) for i in range(inp1c.shape[1])]
vif['Features'] = inp1c.columns
vif.sort_values('VIF',ascending = False)

In [ ]:
# Inference: From the above dataframe we can infer that "service" has high VIF value(which is greter than 5). so we can drop and recheck for VIF

In [ ]:
inp_v1 = inp1.drop('Service',axis = 1)

vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(inp_v1.values,i) for i in range(inp_v1.shape[1])]
vif['Features'] = inp_v1.columns
vif.sort_values('VIF',ascending = False)

In [ ]:
# Inference: From the above dataframe, we can infer that "Others"  having high VIF value which is > 5. so we can drop and recheck for VIF

In [ ]:
inp_v2 = inp_v1.drop('Other (%)',axis = 1)

vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(inp_v2.values,i) for i in range(inp_v2.shape[1])]
vif['Features'] = inp_v2.columns
vif.sort_values('VIF', ascending = False)

In [ ]:
# Inference: From the above dataframe, we can infer that "Infant mortality (per 1000 births)" having high VIF value > 5.
# so we can drop and recheck for VIF.

In [ ]:
inp_v3 = inp_v2.drop('Infant mortality (per 1000 births)',axis = 1)

vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(inp_v3.values,i) for i in range(inp_v3.shape[1])]
vif['Features'] = inp_v3.columns
vif.sort_values('VIF', ascending = False)

In [ ]:
# From the above dataframe we can infer that all the values are < 5. so we can build model to check the significance nature.

In [ ]:
inpc = sm.add_constant(inp_v3)

ols = sm.OLS(out,inpc)
ols_mod = ols.fit()
ols_mod.summary()

In [ ]:
# 2. Normality:

ols_mod.resid.skew()

In [ ]:
ols_mod.resid.plot(kind = 'density')

In [ ]:
# Inference: Above skew value is 0.2, which is in the acceptable range of -0.5 to 0.5. 

# from the above density plot we can conclude that, data is noramlly distributed.

In [ ]:
# 3. Linearity:
from statsmodels.stats.diagnostic import linear_rainbow

In [ ]:
linear_rainbow(res = ols_mod, frac = 0.5)

In [ ]:
# from the above value we can infer that, Pvalue > 0.05. Null hypothesis is accepted, part of data is accepted. 

# Model is statisfying the linearity condition. 

In [ ]:
# 4. Autocorrelation:
ols_mod.summary()

In [ ]:
# from the above constructed ols model as per Durbin-Watson test value is approximately 2.0. Therefore no auto correlation. 

In [ ]:
# 5. Homoscadacity:
inpc = sm.add_constant(inp_v3)

ypred = ols_mod.predict(inpc)

sns.residplot(ypred,ols_mod.resid)

In [ ]:
from statsmodels.stats.api import het_goldfeldquandt

In [ ]:
# H0 : Model is Homoscadasity:

het_goldfeldquandt(ols_mod.resid,inp_v3)

In [ ]:
# p > 0.05. Therefore Null hypothesis hold good. Model is said to be Homoscadasity.

In [ ]:
# Step - 4 ----> Feature Selection:

# 1. Forward Selection
# 2. Backward Elimination
# 3. Recursive Feature Elimination

In [ ]:
pip install mlxtend

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression

In [ ]:
# 1.Forward Selection:
lr = LinearRegression()

lr_forward = sfs(estimator = lr, k_features = 'best', forward = True, scoring = 'r2')

sfs_forward = lr_forward.fit(inp_v3,out)

feat_forward = (sfs_forward.k_feature_names_)
feat_forward = list(feat_forward)

print(feat_forward)
print(sfs_forward.k_score_)

In [ ]:
# 2.Backward Elimination:
lr = LinearRegression()

lr_backward = sfs(estimator = lr, k_features = 'best', forward = False, scoring = 'r2')

sfs_backward = lr_backward.fit(inp_v3,out)

feat_backward = (sfs_backward.k_feature_names_)
feat_backward = list(feat_backward)

print(feat_backward)
print(sfs_backward.k_score_)

In [ ]:
# 3 . RFE method
from sklearn.feature_selection import RFECV

In [ ]:
lr = LinearRegression()

rfecv = RFECV(estimator = lr)
rfe_mod = rfecv.fit(inp_v3,out)
rfe_mod.ranking_

In [ ]:
rank = pd.DataFrame()

rank['Feature'] = inp_v3.columns
rank['Rank'] = rfe_mod.ranking_

feat_rfe = rank[rank['Rank']==1]['Feature']
feat_rfe = list(feat_rfe)
feat_rfe

In [ ]:
# Report card generation to identify the overfitting nature of the model:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error

In [ ]:
# from Forward Selection:
xtrain,xtest,ytrain,ytest = train_test_split(inp_v3,out,test_size = 0.3, random_state = 10)

lr = LinearRegression()
lr.fit(xtrain[feat_forward],ytrain)

ypred_train = lr.predict(xtrain[feat_forward])
ypred_test = lr.predict(xtest[feat_forward])

r2_train = r2_score(ytrain,ypred_train)
r2_test = r2_score(ytest,ypred_test)

mse_train = mean_squared_error(ytrain,ypred_train)
mse_test = mean_squared_error(ytest,ypred_test)

rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

rse_forward = [r2_train,r2_test,rmse_train,rmse_test]
rse_forward

In [ ]:
# from Backward Elimination
xtrain,xtest,ytrain,ytest = train_test_split(inp_v3,out,test_size = 0.3, random_state = 10)

lr = LinearRegression()
lr.fit(xtrain[feat_backward],ytrain)

ypred_train = lr.predict(xtrain[feat_backward])
ypred_test = lr.predict(xtest[feat_backward])

r2_train = r2_score(ytrain,ypred_train)
r2_test = r2_score(ytest,ypred_test)

mse_train = mean_squared_error(ytrain,ypred_train)
mse_test = mean_squared_error(ytest,ypred_test)

rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

rse_backward = [r2_train,r2_test,rmse_train,rmse_test]
rse_backward

In [ ]:
# from RFE
xtrain,xtest,ytrain,ytest = train_test_split(inp_v3,out,test_size = 0.3, random_state = 10)

lr = LinearRegression()
lr.fit(xtrain[feat_rfe],ytrain)

ypred_train = lr.predict(xtrain[feat_rfe])
ypred_test = lr.predict(xtest[feat_rfe])

r2_train = r2_score(ytrain,ypred_train)
r2_test = r2_score(ytest,ypred_test)

mse_train = mean_squared_error(ytrain,ypred_train)
mse_test = mean_squared_error(ytest,ypred_test)

rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

rse_rfe = [r2_train,r2_test,rmse_train,rmse_test]
rse_rfe

In [ ]:
scorecard = pd.DataFrame()
scorecard['Forward'] = rse_forward
scorecard['Backward'] = rse_backward
scorecard['RFE'] = rse_rfe
scorecard.index = ['r2_train','r2_test','rmse_train','rmse_test']
scorecard

In [ ]:
# inference: Difference between r2_train and r2_test is more for RFE by comparing with other two forward and backward. 
#            RFE facing overfitting problem.

In [ ]:
# To analyse Overfitting nature:

# Cross-Validation:
from sklearn.model_selection import cross_val_score

In [ ]:
lr = LinearRegression()

score = cross_val_score(lr, inp_v3, out, cv = 5, scoring = 'r2')

avg_score = np.mean(score)
var_error = np.std(score)
coeff_error = np.std(score) / np.mean(score)

print(avg_score,var_error,coeff_error)

In [ ]:
# inference: Above coeff_error value is nearly 15%, with r-square value of 75 and low bias error.
# so we can conclude that model facing overfitting problem.

In [ ]:
# To overcome the overfitting problem Regularization can be done.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge,Lasso,ElasticNet

In [ ]:
# GridsearchCV using Ridge:

para = {'alpha' : [0.00001,0.0001,0.001,0.01,0.1,0.2,0.5,0.7,1,2,5,10,20,30,50,100]}

rid = Ridge(normalize = True)
grid = GridSearchCV(rid, param_grid = para, scoring = 'r2', cv =5)

mod_grid = grid.fit(xtrain,ytrain)

In [ ]:
# To find best parameter:
mod_grid.best_params_

In [ ]:
# To find best score:
mod_grid.best_score_

In [ ]:
# To get result: based on mean_test_score
pd.DataFrame(mod_grid.cv_results_)

In [ ]:
# To get the result based on std_test_score:
pd.DataFrame(mod_grid.cv_results_).sort_values('std_test_score')

In [ ]:
# GridSearchCV using LASSO:

para = {'alpha' : [0.00001,0.0001,0.001,0.01,0.1,0.2,0.5,0.7,1,2,5,10,20,30,50,100]}

las = Lasso(normalize = True)
grid = GridSearchCV(las, param_grid = para, scoring = 'r2', cv =5)

mod_grid = grid.fit(xtrain,ytrain)

In [ ]:
# To find best Parameter:
mod_grid.best_params_

In [ ]:
# To find best socre:
mod_grid.best_score_

In [ ]:
# To get reuslt:

pd.DataFrame(mod_grid.cv_results_)

In [ ]:
# To get best Standard test score:

pd.DataFrame(mod_grid.cv_results_).sort_values('std_test_score')

In [ ]:
# GridSearchCV using Elasticnet

para = {'alpha' : [0.00001,0.0001,0.001,0.01,0.1,0.2,0.5,0.7,1,2,5,10,20,30,50,100],
        'l1_ratio': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}

ela = ElasticNet(normalize = True)
grid = GridSearchCV(ela, param_grid = para, scoring = 'r2', cv =5)

mod_grid = grid.fit(xtrain,ytrain)

In [ ]:
# To find best parameter:
mod_grid.best_params_

In [ ]:
# To find best score:
mod_grid.best_score_

In [ ]:
# To get result:
pd.DataFrame(mod_grid.cv_results_)

In [ ]:
# To get best standard test score:

pd.DataFrame(mod_grid.cv_results_).sort_values('std_test_score')

In [ ]:
# Overall GridSearchCV inference: # Ridge : 78.85 ; Lasso : 79.39; Elasticnet : 78.86

# By comparing all the three we can conclude that, from Lasso we can get highest R-square value of 79.39 with alpha of 10.

In [ ]:
# Final ols model:
final_inp = inp_v3[feat_forward]
final_inp.head()

In [ ]:
# final OLS Model:

inpc = sm.add_constant(final_inp)

ols = sm.OLS(out.values,inpc)
ols_mod = ols.fit()
ols_mod.summary()

In [ ]:
# Overall model Performance is 79.7%